## Bisección para optimización de thickness y coeficiente de atenuación

### Energía única


In [ ]:
import os
import subprocess
import uproot
import numpy as np

def Bisection(directory, mac_filename, root_filename, tree_name, branch_1, branch_2, initial_energy, thick_0, thick_1, tolerance):
    
    executable_file = "Sim"
    run_sim = f"./{executable_file} {mac_filename} . ."

    ratio = 0
    counter = 1
    counter_2 = 0
    beam_count = 30

    while True:

        thickness = (thick_0 + thick_1) / 2

        mac_template = """\
        /run/numberOfThreads 10
        /run/initialize
        /gun/energy {energy} eV
        /myDetector/ThicknessTarget {thickness:.10f}
        /run/reinitializeGeometry
        /run/beamOn {beam_count}
        """
        
        # create_mac_file
        mac_filepath = os.path.join(directory, mac_filename)
        mac_content = mac_template.format(energy=initial_energy, thickness=thickness, beam_count=beam_count)
        with open(mac_filepath, 'w') as f:
            f.write(mac_content)

        # run_simulation
        try:
            subprocess.run(run_sim, cwd=directory, check=True, shell=True)
        except subprocess.CalledProcessError as e: 
            print(f"Error al ejecutar la simulación: {e}")

        file_path = os.path.join(directory, root_filename)
        if not os.path.isfile(file_path):
            print("Error: El archivo ROOT no existe.")
            break
        
        try:
            with uproot.open(file_path) as root_file:
                tree = root_file[tree_name]
                
                if branch_1 not in tree.keys():
                    print(f"Branch '{branch_1}' not found in tree '{tree_name}' in {file_path}")
                    continue

                hits_count = tree[branch_1].array(library = "np")[0]

        except Exception as e:
            print(f"Error al procesar el archivo ROOT: {e}")
            continue
        
        ratio = hits_count / beam_count * 100

        if   ratio > 50 + (tolerance / 2):
            thick_0 = thickness
        elif ratio < 50 - (tolerance / 2):
            thick_1 = thickness 
        else:
            if counter_2 > 0:
                break
            if counter_2 == 0:
                beam_count = 10000
                counter_2 = 1

        counter = counter + 1
        if counter == 35:
            print("No se encontró una solución en el rango especificado.")
            break

    try:
        with uproot.open(file_path) as root_file:
            tree = root_file[tree_name]
            
            if branch_2 in tree.keys():
                coeficient = tree[branch_2].array(library="np")[0]  # Assuming you want the first entry
                print('Coeficiente de atenuación:', coeficient)
            else:
                print(f"Branch '{branch_2}' not found in tree '{tree_name}' in {file_path}")
    
    except Exception as e:
        print(f"Error al procesar el archivo ROOT: {e}")

    print('Ratio:', ratio)
    print("Número de iteraciones:", counter)
    print(f"Optimización completada: Thickness óptimo = {thickness} mm")

In [ ]:
directory = 'BUILD/'

mac_filename = 'Bisection.mac'
root_filename = "ROOT/root0.root"

tree_name = 'Transportation'
branch_1 = 'Ratio'
branch_2 = 'Mass_Attenuation'

initial_energy = 50 * 1000  # eV

thickness = 1
thick_0 = thickness / 1e3  # mm 
thick_1 = thickness * 1e3  # mm

tolerance = 30

Bisection(directory, mac_filename, root_filename, tree_name, branch_1, branch_2, initial_energy, thick_0, thick_1, tolerance)

### Múltiples energías

In [ ]:
import os
import subprocess
import pandas as pd
import uproot

def Energies(directory, mac_filename, root_filename, outputcsv_name, tree_name, branch_1, branch_2, initial_energy, final_energy, energy_step, thickness, tolerance):
    
    executable_file = "Sim"
    run_sim = f"./{executable_file} {mac_filename} . ."
    output_file = os.path.join(directory + 'ROOT/' + outputcsv_name)

    results = []

    for energy in range(initial_energy, final_energy, energy_step):

        ratio = 0
        counter_1 = 1
        counter_2 = 0
        beam_count = 200

        thickness = 0.1 * initial_energy / 1000 # mm

        thick_0 = thickness / 1e3  # mm 
        thick_1 = thickness * 1e3  # mm

        while True:

            thickness = (thick_0 + thick_1) / 2

            mac_template = """\
            /run/numberOfThreads 1
            /run/initialize
            /gun/energy {energy} eV
            /myDetector/ThicknessTarget {thickness:.12f}
            /run/reinitializeGeometry
            /run/beamOn {beam_count}
            """

            # create_mac_file
            mac_filepath = os.path.join(directory, mac_filename)
            mac_content = mac_template.format(energy=energy, thickness=thickness, beam_count=beam_count)
            with open(mac_filepath, 'w') as f:
                f.write(mac_content)

            # run_simulation
            try:
                subprocess.run(run_sim, cwd=directory, check=True, shell=True, stdout=subprocess.DEVNULL)
            
            except subprocess.CalledProcessError as e: 
                print(f"Error al ejecutar la simulación: {e}")

            # count_events_from_root using uproot
            file_path = os.path.join(directory, root_filename)
            if not os.path.isfile(file_path):
                print("Error: El archivo ROOT no existe.")
                break
            
            try:
                with uproot.open(file_path) as root_file:
                    tree = root_file[tree_name]
                    if branch_1 not in tree.keys():
                        print(f"Branch '{branch_1}' not found in tree '{tree_name}' in {file_path}")
                        continue

                    hits_count = tree[branch_1].array(library="np")[0]  # Assuming you want the first entry

            except Exception as e:
                print(f"Error al procesar el archivo ROOT: {e}")
                continue
            
            ratio = hits_count / beam_count * 100

            print(ratio)

            if   ratio > 50 + (tolerance / 2):
                thick_0 = thickness
            elif ratio < 50 - (tolerance / 2):
                thick_1 = thickness 
            
            else:

                print('rango')

                if counter_2 > 0:
                    
                    try:
                        branch2_array = tree[branch_2].array(library="np")
                        if len(branch2_array) > 0:
                            coeficient = branch2_array[0]
                            results.append({'Energy': energy / 1000, 'Optimal_Thickness': thickness, 'AtCoefficient': coeficient})
                            print('Energía calculada:', energy)
                        else:
                            print(f"No data in branch '{branch_2}' in tree '{tree_name}' in {file_path}")
                    except Exception as e:
                        print(f"Error al procesar el branch '{branch_2}': {e}")
                    
                    break

                if counter_2 == 0:
                    beam_count = 10000
                    counter_2 = 1

            counter_1 += 1
            if counter_1 == 30:
                print("No se encontró una solución en el rango especificado.")
                break

    results_df = pd.DataFrame(results)
    results_df.to_csv(output_file, index=False)

In [ ]:
directory = 'BUILD/'

mac_filename = 'Bisection.mac'
root_filename = "ROOT/root0.root"
outputcsv_name = 'Aluminio2.csv'

tree_name = 'Transportation'
branch_1 = 'Ratio'
branch_2 = 'Mass_Attenuation'

initial_energy = 50 * 1000 # eV
final_energy = 52 * 1000  # eV
energy_step = 1 * 1000

thickness = 1 # mm

tolerance = 10

Energies(directory, mac_filename, root_filename, outputcsv_name, tree_name, branch_1, branch_2, initial_energy, final_energy, energy_step, thickness, tolerance)

### Plot CSV (X axis vs. Y axis)

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_xy(directory, csv_name, x_branch, y_branch, title, x_label, y_label):

    csv_file = os.path.join(directory, csv_name)
    df = pd.read_csv(csv_file)

    x = df[x_branch]
    y = df[y_branch]

    SIZE_DEFAULT = 14
    SIZE_LARGE = 20

    plt.rc("font", family = 'Century Expanded')  
    plt.rc("font", weight = "normal")  
    plt.rc("font",  size      = SIZE_DEFAULT)  
    plt.rc("axes",  titlesize = SIZE_LARGE + 2)  
    plt.rc("axes",  labelsize = SIZE_LARGE)  
    plt.rc("xtick", labelsize = SIZE_DEFAULT)  
    plt.rc("ytick", labelsize = SIZE_DEFAULT)  

    plt.figure(figsize = (10, 6))
    plt.tight_layout(rect = [0, 0.01, 1, 1])

    plt.plot(x, y, marker = 'o')
    plt.grid(True, alpha = 0.5)
    
    plt.xlabel(x_label, fontweight = "bold", labelpad = 21)
    plt.ylabel(y_label, fontweight = "bold", labelpad = 22)
    plt.title (title  , fontweight = "bold", pad = 25)
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['top'].set_alpha(0.3)
    ax.spines['right'].set_alpha(0.3)
    ax.spines['bottom'].set_alpha(0.7)
    ax.spines['left'].set_alpha(0.7)

    plt.savefig(title + '.png', dpi = 10)
    plt.show()

    print('Plot saved as', title + '.png')

In [ ]:
directory = 'build/root/'
csv_name = 'Aluminio2.csv'

x_branch = "Energy"
y_branch = 'Optimal_Thickness'

title = "title"
x_label = r"x axis ($keV$)"
y_label = r"y axis"

plot_xy(directory, csv_name, x_branch, y_branch, title, x_label, y_label)

### Graficar Real vs Calculado

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

def plot_xy(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label):

    path_1 = os.path.join(directory, csv_1)
    path_2 = os.path.join(directory, csv_2)
    df_1 = pd.read_csv(path_1)
    df_2 = pd.read_csv(path_2)

    x1 = df_1[x_branch_1]
    y1 = df_1[y_branch_1]

    x2 = df_2[x_branch_2]
    y2 = df_2[y_branch_2]

    SIZE_DEFAULT = 14
    SIZE_LARGE = 20

    plt.rc("font", family = 'Century Expanded')  
    plt.rc("font", weight = "normal")  
    plt.rc("font",  size      = SIZE_DEFAULT)  
    plt.rc("axes",  titlesize = SIZE_LARGE + 2)  
    plt.rc("axes",  labelsize = SIZE_LARGE)  
    plt.rc("xtick", labelsize = SIZE_DEFAULT)  
    plt.rc("ytick", labelsize = SIZE_DEFAULT)  

    plt.figure(figsize = (10, 6))
    plt.tight_layout(rect = [0, 0.01, 1, 1])

    plt.plot(x1, y1, marker = 'o', markersize = 1)
    plt.plot(x2, y2, marker = 'o', markersize = 1)
    plt.grid(True, alpha = 0.5)

    plt.xscale('log')
    plt.yscale('log')   
    
    plt.xlabel(x_label, fontweight = "bold", labelpad = 21)
    plt.ylabel(y_label, fontweight = "bold", labelpad = 22)
    plt.title (title  , fontweight = "bold", pad = 25)
    
    ax = plt.gca()
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.spines['bottom'].set_visible(True)
    ax.spines['left'].set_visible(True)
    ax.spines['top'].set_alpha(0.3)
    ax.spines['right'].set_alpha(0.3)
    ax.spines['bottom'].set_alpha(0.7)
    ax.spines['left'].set_alpha(0.7)

    plt.savefig(title + '.png', dpi = 400)
    plt.show()

    print('Plot saved as', title + '.png')

In [ ]:
directory = 'BUILD/ROOT/'

csv_1 = 'Al_real.csv'
x_branch_1 = "Energy"
y_branch_1 = 'AtCoefficient'

csv_2 = 'Al_calc.csv'
x_branch_2 = "Energy"
y_branch_2 = 'AtCoefficient'

title = "Coefficient vs Energy"
x_label = r"Energy ($KeV$)"
y_label = r"Thicckness ($mm$)"

plot_xy(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label)

### Gráfica Logarítmica JP

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

def plot_jp(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label):

    path_1 = os.path.join(directory, csv_1)
    path_2 = os.path.join(directory, csv_2)
    
    df_1 = pd.read_csv(path_1)
    df_2 = pd.read_csv(path_2)

    x1 = df_1[x_branch_1]
    y1 = df_1[y_branch_1]

    x2 = df_2[x_branch_2]
    y2 = df_2[y_branch_2]

    # # Merge the DataFrames on the energy column
    # merged_df = pd.merge(calc_df, real_df, on=energy_col, suffixes=('_calc', '_real'))
    # # Calculate global percentage error
    # def calculate_percentage_error(row):
    #     if row[f'{value_col}_real'] == 0:
    #         return np.nan  # Avoid division by zero
    #     return abs((row[f'{value_col}_calc'] - row[f'{value_col}_real']) / row[f'{value_col}_real']) * 100
    # merged_df['Percentage_Error'] = merged_df.apply(calculate_percentage_error, axis=1)
    # # Calculate global percentage error
    # global_percentage_error = merged_df['Percentage_Error'].mean()
    # print(f"Global Percentage Error: {global_percentage_error:.2f}%")

    plt.figure(figsize=(10, 6))
    plt.grid(True, which='both', linestyle='--', linewidth=0.7)

    plt.plot(x1, y1, marker = 'o', markersize = 2, label='Calculated', color='blue')
    plt.plot(x2, y2, marker = 'x', markersize = 2, label='Real', color='red')

    plt.xscale('log')
    plt.yscale('log')

    plt.xlabel('Energy (keV)')
    plt.ylabel('LACm')
    plt.title('Massic Linear Atenuation Coefficient (W)')
    plt.legend()

    # plt.figtext(0.15, 0.85, f'Global Percentage Error: {global_percentage_error:.2f}%', fontsize=12, bbox=dict(facecolor='white', alpha=0.5))

    plt.savefig(title + '.png', dpi = 400)
    plt.show()

    print('Plot saved as', title + '.png')
    

In [ ]:
directory = 'BUILD/ROOT/'

csv_1 = 'Al_real.csv'
x_branch_1 = "Energy"
y_branch_1 = 'AtCoefficient'

csv_2 = 'Al_calc.csv'
x_branch_2 = "Energy"
y_branch_2 = 'AtCoefficient'

title = "Coefficient vs Energy"
x_label = r"Energy ($KeV$)"
y_label = r"Thicckness ($mm$)"

plot_jp(directory, csv_1, x_branch_1, y_branch_1, csv_2, x_branch_2, y_branch_2, title, x_label, y_label)